In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This code was used to get the correct sets for train, validation and test based on the amount of frames to be used. This code was run four times, for 16 frames, 32 frames, 48 frames and for the full video length. The code reduces the frames by skipping every 2 or 3 frames to reduce the length to 16 frames while keeping the temporal aspect of 32, 48 or full video.



In [ ]:
import os
import cv2
import numpy as np
import concurrent.futures

folder_path = 'drive/MyDrive/Colab Notebooks/Approach_100'
training_path = 'drive/MyDrive/Colab Notebooks/frames/train56'
validation_path = 'drive/MyDrive/Colab Notebooks/frames/validation56'
test_path = 'drive/MyDriveframes/Colab Notebooks/test56'

get_percentage = False
frame_num = 56 #change to desired number of frames


def get_all_mp4_files(folder):
    # Get a list of all files in the directory
    file_list = os.listdir(folder)

    # Filter the list to include only .mp4 files
    mp4_files = [f for f in file_list if f.endswith('.mp4')]

    return mp4_files


def split_videos_in_sets(mp4_files):
    train_set = []
    val_set = []
    test_set = []

    for file in mp4_files:
        # Extract the substring from the file name
        first_number = int(file.split('-')[0].replace('d', ''))

        if first_number >= 20:
            test_set.append(file)
        elif first_number >= 18:
            val_set.append(file)
        else:
            train_set.append(file)

    return train_set, val_set, test_set


def get_frames_to_extract(frame_count, num_frames):
    if frame_count < 16:
        print(f"Error too little frames! It will duplicate some frames to reach 16 frames.")

    values = np.linspace(0, (frame_count - 1), num_frames)
    return np.array([round(i) for i in values])


def preprocess_video_steps(video_path, output_path, num_frames, get_percentage, frame_count):
    # Initialize an empty list to store frames
    frames = []

    # Open the video file using OpenCV
    video = cv2.VideoCapture(video_path)

    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if get_percentage:
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    elif int(video.get(cv2.CAP_PROP_FRAME_COUNT)) < frame_count:
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    frames_to_extract = get_frames_to_extract(frame_count, num_frames)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, video.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

    # Initialize the frame index
    frame_index = 0
    frame_extracted = 0

    while True:
        # Read the next frame from the video
        ret, frame = video.read()

        # Check if there are no more frames to read
        if not ret:
            break

        # Check if we have processed the desired number of frames
        if frame_extracted == num_frames:
            break

        if frame_index in frames_to_extract:
            out.write(frame)
            frame_extracted += 1

        # Increment the frame index
        frame_index += 1

    video.release()

    zero_padded = 0
    # Zero pad frames to num_frames if necessary
    while frame_extracted < num_frames:
        # Create a black frame
        black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
        out.write(black_frame)
        frame_extracted += 1
        zero_padded += 1

    out.release()
    print(f"Looped over {frame_index} frames for {video_path} with {zero_padded} zeropad")
    return frames


def get_frame_count(file):
    cap = cv2.VideoCapture(file)
    return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


def process_file(file, input_dir, output_dir, num_frames, get_percentage, frame_count):
    vid_path = os.path.join(input_dir, file)
    out_path = os.path.join(output_dir, file)
    preprocess_video_steps(vid_path, out_path, num_frames, get_percentage, frame_count)
    if get_frame_count(out_path) != num_frames:
        print(f"Incorrect frame count for: {out_path}")


def get_frames_for_files(files, input_dir, output_dir, get_percentage=True, frame_count=48, num_frames=16):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(lambda file: process_file(file, input_dir, output_dir, num_frames, get_percentage, frame_count),
                     files)


files = get_all_mp4_files(folder_path)
train_set, val_set, test_set = split_videos_in_sets(files)

get_frames_for_files(train_set, folder_path, training_path, get_percentage, frame_num)
get_frames_for_files(val_set, folder_path, validation_path, get_percentage, frame_num)
get_frames_for_files(test_set, folder_path, test_path, get_percentage, frame_num)


In [3]:
#check framelength for an orginally 8-second video

#file d7-rB-s4-a7_100.mp4 is orginally 8.78 seconds

import cv2

def count_frames(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Couldn't open the video file.")
        return -1

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return total_frames

# Example usage:
video_file_path = '/content/drive/MyDrive/Colab Notebooks /Approach_100/d7-rB-s4-a7_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
    print(f"Total frames in '{video_file_path}': {frame_count}")

#16frames
video_file_path = '/content/drive/MyDrive/Colab Notebooks /frames/train16/d7-rB-s4-a7_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
    print(f"Total frames in '{video_file_path}': {frame_count}")

#32frames
video_file_path = '/content/drive/MyDrive/Colab Notebooks /frames/train32/d7-rB-s4-a7_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
    print(f"Total frames in '{video_file_path}': {frame_count}")
#48frames

video_file_path = '/content/drive/MyDrive/Colab Notebooks /frames/train48/d1-rB-s4-a8_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
  print(f"Total frames in '{video_file_path}': {frame_count}")

#56frames

video_file_path = '/content/drive/MyDrive/Colab Notebooks /frames/train64/d1-rB-s4-a8_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
  print(f"Total frames in '{video_file_path}': {frame_count}")

#full_set

video_file_path = '/content/drive/MyDrive/Colab Notebooks /frames/train_full/d1-rB-s4-a8_100.mp4'  # Replace with your video file path
frame_count = count_frames(video_file_path)

if frame_count != -1:
  print(f"Total frames in '{video_file_path}': {frame_count}")




Total frames in '/content/drive/MyDrive/Colab Notebooks /Approach_100/d7-rB-s4-a7_100.mp4': 263
Total frames in '/content/drive/MyDrive/Colab Notebooks /frames/train16/d7-rB-s4-a7_100.mp4': 16
Total frames in '/content/drive/MyDrive/Colab Notebooks /frames/train32/d7-rB-s4-a7_100.mp4': 16
Total frames in '/content/drive/MyDrive/Colab Notebooks /frames/train48/d1-rB-s4-a8_100.mp4': 16
Total frames in '/content/drive/MyDrive/Colab Notebooks /frames/train64/d1-rB-s4-a8_100.mp4': 16
Total frames in '/content/drive/MyDrive/Colab Notebooks /frames/train_full/d1-rB-s4-a8_100.mp4': 16
